In [ ]:
from utils import mkdir_no_exist
import os
import sys
import subprocess

import pandas as pd
import numpy as np
import json

In [ ]:
current_dir = os.path.abspath("")
sys.path.append(current_dir)

In [ ]:
dataset_id = 'ds000171'
subjects = ['sub-control{:02d}'.format(i+1) for i in range(20)]

dataset_path = os.path.join(current_dir, "data", dataset_id)
deriv_path = os.path.join(current_dir,"data", "derivatives")
preproc_path = os.path.join(deriv_path, 'preprocessed_data')

mkdir_no_exist(dataset_path)
mkdir_no_exist(preproc_path)

# download files yourself from google drive into dataset_path
